In [1]:
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings,HuggingFaceEndpoint
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llms=HuggingFaceEndpoint(
    repo_id="google-gemma-2b-2-it",
    task="text-generation"
)

In [6]:
# Load docs
loader = PyPDFLoader("ML_Book.pdf")
docs = loader.load()

In [7]:
# Split
split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = split.split_documents(docs)

In [9]:
# Embeddings + store
embedding=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
db = FAISS.from_documents(chunks, embedding)

In [10]:

# Retriever
retriever = db.as_retriever()

In [11]:

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use the following data:
{context}

Question: {question}
Answer:"""
)

In [12]:
llm = ChatHuggingFace(
    llm=llms
)


In [14]:
from langchain_core.runnables import RunnablePassthrough

In [15]:

# Pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke()